# 🚀 Tam Otomatik JetX Model Eğitmeni (v3.0)

Bu not defteri, JetX tahmin sistemi için yeniden yapılandırılmış ve tam otomatik bir eğitim akışı sağlar.

## ✨ Yeni Özellikler (v3.0)

- **Otomatik Hiperparametre Optimizasyonu (HPO):** `Optuna` entegrasyonu ile her model için en iyi hiperparametreler otomatik olarak bulunur.
- **Deney Takibi:** `MLflow` entegrasyonu ile tüm eğitim süreçleri, metrikler ve modeller kaydedilir ve izlenebilir.
- **Merkezi Yapılandırma:** Tüm ayarlar artık proje kök dizinindeki `config.yaml` dosyasından yönetilir.
- **Kod Standardizasyonu:** Proje tamamen `PyTorch` üzerine odaklanmış, eski ve kullanılmayan kodlar temizlenmiştir.

**⚠️ Başlamadan Önce: Runtime > Change runtime type > GPU seçeneğini etkinleştirin!**

## 🛠️ 1. Sistem Kurulumu ve Projeyi Yükleme

In [ ]:
import os
import sys
import subprocess

def setup_environment():
    """Clones or updates the repository and sets up the environment."""
    repo_url = "https://github.com/onndd/predictor_1.git"
    project_dir = "/content/predictor_1"

    if os.path.exists(project_dir):
        print("📁 Proje zaten mevcut. Güncelleniyor...")
        os.chdir(project_dir)
        subprocess.run(["git", "pull", "origin", "main"], check=True, capture_output=True)
    else:
        print("📥 GitHub'dan proje klonlanıyor...")
        os.chdir("/content")
        subprocess.run(["git", "clone", repo_url], check=True, capture_output=True)
        os.chdir(project_dir)

    # Add src to Python path
    src_path = os.path.join(project_dir, "src")
    if src_path not in sys.path:
        sys.path.insert(0, src_path)
        print(f"✅ {src_path} Python path'e eklendi")

    # Install dependencies
    requirements_path = os.path.join(project_dir, 'requirements_enhanced.txt')
    if os.path.exists(requirements_path):
        print("📦 Gerekli kütüphaneler yükleniyor...")
        # --quiet bayrağı kaldırıldı
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-r", requirements_path])
        print("✅ Kütüphaneler başarıyla yüklendi.")

setup_environment()
print("🎉 Kurulum tamamlandı!")

## 📊 2. MLflow Deney Takibini Başlatma

Bu hücre, MLflow arayüzünü başlatır ve `ngrok` kullanarak size erişebileceğiniz bir public URL verir. Eğitim metriklerini ve sonuçlarını bu arayüzden takip edebilirsiniz.

In [ ]:
# ngrok kurulumu
!pip install pyngrok --quiet

# MLflow UI'ı arka planda başlat
get_ipython().system_raw('mlflow ui --host 0.0.0.0 &')

# ngrok ile public URL oluştur
from pyngrok import ngrok
print("✅ MLflow arayüzü için public URL oluşturuluyor...")
print("👇 Aşağıdaki linke tıklayarak deneyleri izleyebilirsiniz:")
print(ngrok.connect(5000))

## 🚀 3. Otomatik Eğitim Sürecini Başlatma

In [ ]:
import torch
from src.training.master_trainer import MasterTrainer
import traceback

# Cihaz tespiti (GPU veya CPU)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device == 'cuda':
    print(f"✅ GPU bulundu: {torch.cuda.get_device_name(0)}")
else:
    print("⚠️ GPU bulunamadı. Eğitim CPU ile devam edecek.")

try:
    # MasterTrainer'ı başlat ve tüm eğitim sürecini çalıştır.
    # Bu işlem HPO, eğitim ve MLflow kaydını içerir.
    master_trainer = MasterTrainer(models_to_train=['N-Beats', 'TFT', 'LSTM'], device=device)
    master_trainer.run()
except Exception as e:
    print(f"❌ Eğitim sırasında beklenmedik bir hata oluştu: {e}")
    traceback.print_exc()